## Wide & Deep NN

In [8]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import pandas as pd

2.13.0


In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [14]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
X_new = X_test[:3]

In [17]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 30)                   270       ['input_2[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 30)                   930       ['dense_7[0][0]']             
                                                                                                  
 concatenate_1 (Concatenate  (None, 38)                   0         ['input_2[0][0]',             
 )                                                                   'dense_8[0][0]']       

In [19]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 0s 574us/step - loss: 1.9430 - val_loss: 0.9191
Epoch 2/20
363/363 [==============================] - 0s 444us/step - loss: 0.8670 - val_loss: 0.7768
Epoch 3/20
363/363 [==============================] - 0s 430us/step - loss: 0.7465 - val_loss: 0.6985
Epoch 4/20
363/363 [==============================] - 0s 454us/step - loss: 0.6730 - val_loss: 0.6586
Epoch 5/20
363/363 [==============================] - 0s 452us/step - loss: 0.6340 - val_loss: 0.6272
Epoch 6/20
363/363 [==============================] - 0s 474us/step - loss: 0.6044 - val_loss: 0.6047
Epoch 7/20
363/363 [==============================] - 0s 461us/step - loss: 0.5819 - val_loss: 0.5827
Epoch 8/20
363/363 [==============================] - 0s 446us/step - loss: 0.5628 - val_loss: 0.5672
Epoch 9/20
363/363 [==============================] - 0s 448us/step - loss: 0.5484 - val_loss: 0.5543
Epoch 10/20
363/363 [==============================] - 0s 432us/step - loss: 0.536

In [20]:
y_pred

array([[0.7401556],
       [1.4516754],
       [2.7926033]], dtype=float32)

와이드나 딥 경로에 다른 입력 특성을 전달하면 어떻게 될까요? (특성 0에서 4까지) 5개의 특성을 와이드 경로에 보내고 (특성 2에서 7까지) 6개의 특성을 딥 경로에 전달하겠습니다. 3개의 특성(특성 2, 3, 4)은 양쪽에 모두 전달됩니다.

In [23]:
np.random.seed(42)
tf.random.set_seed(42)

In [25]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [26]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
# history = model.fit({"wide_input": X_train_A, "deep_input": X_train_B}, y_train, epochs=20, validation_data=({"wide_input": X_valid_A, "deep_input": X_valid_B}, y_valid))
# 딕셔너리 형태로 명시해줄 수 있음

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 0s 618us/step - loss: 1.9893 - val_loss: 0.9051
Epoch 2/20
363/363 [==============================] - 0s 442us/step - loss: 0.7966 - val_loss: 0.7065
Epoch 3/20
363/363 [==============================] - 0s 453us/step - loss: 0.6626 - val_loss: 0.6378
Epoch 4/20
363/363 [==============================] - 0s 654us/step - loss: 0.6091 - val_loss: 0.6033
Epoch 5/20
363/363 [==============================] - 0s 454us/step - loss: 0.5792 - val_loss: 0.5793
Epoch 6/20
363/363 [==============================] - 0s 455us/step - loss: 0.5597 - val_loss: 0.5671
Epoch 7/20
363/363 [==============================] - 0s 541us/step - loss: 0.5459 - val_loss: 0.5515
Epoch 8/20
363/363 [==============================] - 0s 508us/step - loss: 0.5349 - val_loss: 0.5418
Epoch 9/20
363/363 [==============================] - 0s 462us/step - loss: 0.5265 - val_loss: 0.5352
Epoch 10/20
363/363 [==============================] - 0s 447us/step - loss: 0.520

In [27]:
y_pred

array([[0.61095685],
       [1.7675967 ],
       [2.980628  ]], dtype=float32)

### 출력이 여러 개 필요할 때
- 동일한 데이터에서 독립적인 여러 작업을 수행할 때
- 규제 기법으로 사용하는 경우

In [28]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [32]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-3))

In [33]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 846us/step - loss: 2.6756 - main_output_loss: 2.4743 - aux_output_loss: 4.4875 - val_loss: 1.2355 - val_main_output_loss: 0.9947 - val_aux_output_loss: 3.4023
Epoch 2/20
363/363 [==============================] - 0s 528us/step - loss: 0.9995 - main_output_loss: 0.8040 - aux_output_loss: 2.7587 - val_loss: 0.8584 - val_main_output_loss: 0.7007 - val_aux_output_loss: 2.2770
Epoch 3/20
363/363 [==============================] - 0s 517us/step - loss: 0.8018 - main_output_loss: 0.6715 - aux_output_loss: 1.9742 - val_loss: 0.7600 - val_main_output_loss: 0.6484 - val_aux_output_loss: 1.7638
Epoch 4/20
363/363 [==============================] - 0s 526us/step - loss: 0.7283 - main_output_loss: 0.6313 - aux_output_loss: 1.6012 - val_loss: 0.7103 - val_main_output_loss: 0.6239 - val_aux_output_loss: 1.4881
Epoch 5/20
363/363 [==============================] - 0s 532us/step - loss: 0.6871 - main_output_loss: 0.6087 - aux_output_loss: 1.3925 

In [34]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

162/162 [==============================] - 0s 396us/step - loss: 0.5469 - main_output_loss: 0.5145 - aux_output_loss: 0.8379


1/1 [==============================] - 0s 26ms/step


In [37]:
print(y_pred_main)
print()
print(y_pred_aux)

[[0.7054903]
 [1.759553 ]
 [2.7266278]]

[[1.1412153]
 [1.844955 ]
 [1.9123092]]


## Subclassing API 로 동적 모델 만들기

In [38]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # 표준 매개변수 처리 (예를 들면, name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel(30, activation="relu")

In [40]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 0s 802us/step - loss: 2.6636 - output_1_loss: 2.4449 - output_2_loss: 4.6319 - val_loss: 1.3895 - val_output_1_loss: 1.0941 - val_output_2_loss: 4.0486
Epoch 2/10
363/363 [==============================] - 0s 679us/step - loss: 1.2109 - output_1_loss: 0.9581 - output_2_loss: 3.4868 - val_loss: 0.9503 - val_output_1_loss: 0.7230 - val_output_2_loss: 2.9959
Epoch 3/10
363/363 [==============================] - 0s 513us/step - loss: 0.8798 - output_1_loss: 0.6872 - output_2_loss: 2.6131 - val_loss: 0.8399 - val_output_1_loss: 0.6716 - val_output_2_loss: 2.3545
Epoch 4/10
363/363 [==============================] - 0s 518us/step - loss: 0.7979 - output_1_loss: 0.6514 - output_2_loss: 2.1161 - val_loss: 0.7799 - val_output_1_loss: 0.6484 - val_output_2_loss: 1.9631
Epoch 5/10
363/363 [==============================] - 0s 529us/step - loss: 0.7453 - output_1_loss: 0.6279 - output_2_loss: 1.8017 - val_loss: 0.7381 - val_output_1_loss: 0.629

1/1 [==============================] - 0s 25ms/step


### 모델 저장과 복원

In [41]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
]) 

In [42]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 0s 585us/step - loss: 1.9216 - val_loss: 0.9798
Epoch 2/10
363/363 [==============================] - 0s 640us/step - loss: 0.8279 - val_loss: 0.7546
Epoch 3/10
363/363 [==============================] - 0s 471us/step - loss: 0.7092 - val_loss: 0.6915
Epoch 4/10
363/363 [==============================] - 0s 454us/step - loss: 0.6562 - val_loss: 0.6509
Epoch 5/10
363/363 [==============================] - 0s 452us/step - loss: 0.6244 - val_loss: 0.6153
Epoch 6/10
363/363 [==============================] - 0s 449us/step - loss: 0.5874 - val_loss: 0.5851
Epoch 7/10
363/363 [==============================] - 0s 433us/step - loss: 0.5595 - val_loss: 0.5620
Epoch 8/10
363/363 [==============================] - 0s 490us/step - loss: 0.5374 - val_loss: 0.5425
Epoch 9/10
363/363 [==============================] - 0s 463us/step - loss: 0.5211 - val_loss: 0.5255
Epoch 10/10
162/162 [==============================] - 0s 310us/step - loss: 0.502

In [43]:
model.save("my_keras_model.h5")

/Users/ldywin/PycharmProjects/MLDLPractice/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
model = keras.models.load_model("my_keras_model.h5")

In [45]:
model.predict(X_new)

1/1 [==============================] - 0s 26ms/step


array([[0.7380551],
       [1.3488903],
       [2.8796465]], dtype=float32)

In [46]:
model.save_weights("my_keras_weights.ckpt")

In [47]:
model.load_weights("my_keras_weights.ckpt")

### Using callback during fit()

In [48]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [49]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [50]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])
model = keras.models.load_model("my_keras_model.h5") # 최상의 모델로 롤백
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 0s 597us/step - loss: 2.3563 - val_loss: 1.0113
Epoch 2/10
363/363 [==============================] - 0s 439us/step - loss: 0.8437 - val_loss: 0.7195
Epoch 3/10
  1/363 [..............................] - ETA: 0s - loss: 0.7692

/Users/ldywin/PycharmProjects/MLDLPractice/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


363/363 [==============================] - 0s 452us/step - loss: 0.6750 - val_loss: 0.6589
Epoch 4/10
363/363 [==============================] - 0s 453us/step - loss: 0.6249 - val_loss: 0.6239
Epoch 5/10
363/363 [==============================] - 0s 449us/step - loss: 0.5917 - val_loss: 0.5946
Epoch 6/10
363/363 [==============================] - 0s 440us/step - loss: 0.5637 - val_loss: 0.5708
Epoch 7/10
363/363 [==============================] - 0s 470us/step - loss: 0.5406 - val_loss: 0.5490
Epoch 8/10
363/363 [==============================] - 0s 438us/step - loss: 0.5209 - val_loss: 0.5315
Epoch 9/10
363/363 [==============================] - 0s 445us/step - loss: 0.5049 - val_loss: 0.5175
Epoch 10/10
162/162 [==============================] - 0s 304us/step - loss: 0.4918


In [51]:
mse_test

0.49176082015037537

In [52]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
363/363 [==============================] - 0s 605us/step - loss: 0.4811 - val_loss: 0.4970
Epoch 2/100
363/363 [==============================] - 0s 454us/step - loss: 0.4725 - val_loss: 0.4872
Epoch 3/100
363/363 [==============================] - 0s 461us/step - loss: 0.4655 - val_loss: 0.4813
Epoch 4/100
363/363 [==============================] - 0s 732us/step - loss: 0.4602 - val_loss: 0.4763
Epoch 5/100
363/363 [==============================] - 0s 459us/step - loss: 0.4555 - val_loss: 0.4710
Epoch 6/100
363/363 [==============================] - 0s 506us/step - loss: 0.4512 - val_loss: 0.4701
Epoch 7/100
363/363 [==============================] - 0s 470us/step - loss: 0.4482 - val_loss: 0.4639
Epoch 8/100
363/363 [==============================] - 0s 465us/step - loss: 0.4448 - val_loss: 0.4614
Epoch 9/100
363/363 [==============================] - 0s 463us/step - loss: 0.4421 - val_loss: 0.4588
Epoch 10/100
363/363 [==============================] - 0s 448us/step - l

In [53]:
mse_test

0.3583745062351227

사용자 정의 콜백: 검증 손실과 훈련 손실의 비율을 출력함으로써 과대적합 감지

In [55]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [57]:
val_train_ratio_cb = PrintValTrainRatioCallback()
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb])

272/363 [=====================>........] - ETA: 0s - loss: 0.3459
val/train: 1.09
363/363 [==============================] - 0s 520us/step - loss: 0.3436 - val_loss: 0.3732


## 텐서보드 사용

In [58]:
import os

In [59]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [60]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2023_07_31-21_48_00'

In [61]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [62]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [63]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 0s 711us/step - loss: 1.8568 - val_loss: 0.9101
Epoch 2/30
363/363 [==============================] - 0s 544us/step - loss: 0.7530 - val_loss: 0.6777
Epoch 3/30
363/363 [==============================] - 0s 524us/step - loss: 0.6463 - val_loss: 0.6344
Epoch 4/30
363/363 [==============================] - 0s 539us/step - loss: 0.6070 - val_loss: 0.6020
Epoch 5/30
363/363 [==============================] - 0s 526us/step - loss: 0.5772 - val_loss: 0.5749
Epoch 6/30
363/363 [==============================] - 0s 517us/step - loss: 0.5520 - val_loss: 0.5538
Epoch 7/30
363/363 [==============================] - 0s 523us/step - loss: 0.5309 - val_loss: 0.5355
Epoch 8/30
363/363 [==============================] - 0s 514us/step - loss: 0.5136 - val_loss: 0.5199
Epoch 9/30
363/363 [==============================] - 0s 504us/step - loss: 0.5000 - val_loss: 0.5082
Epoch 10/30
363/363 [==============================] - 0s 503us/step - loss: 0.489

In [64]:
run_logdir2 = get_run_logdir()
run_logdir2

'./my_logs/run_2023_07_31-22_10_59'

In [65]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [66]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=0.05))

In [67]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir2)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 0s 635us/step - loss: 0.5841 - val_loss: 0.4405
Epoch 2/30
363/363 [==============================] - 0s 646us/step - loss: 0.4386 - val_loss: 0.5421
Epoch 3/30
363/363 [==============================] - 0s 558us/step - loss: nan - val_loss: nan
Epoch 4/30
363/363 [==============================] - 0s 531us/step - loss: nan - val_loss: nan
Epoch 5/30
363/363 [==============================] - 0s 537us/step - loss: nan - val_loss: nan
Epoch 6/30
363/363 [==============================] - 0s 514us/step - loss: nan - val_loss: nan
Epoch 7/30
363/363 [==============================] - 0s 598us/step - loss: nan - val_loss: nan
Epoch 8/30
363/363 [==============================] - 0s 555us/step - loss: nan - val_loss: nan
Epoch 9/30
363/363 [==============================] - 0s 555us/step - loss: nan - val_loss: nan
Epoch 10/30
363/363 [==============================] - 0s 493us/step - loss: nan - val_loss: nan
Epoch 11/30
363/363 [======

사용할 수 있는 로깅 옵션 확인

In [69]:
help(keras.callbacks.TensorBoard.__init__)

Help on function __init__ in module keras.src.callbacks:

__init__(self, log_dir='logs', histogram_freq=0, write_graph=True, write_images=False, write_steps_per_second=False, update_freq='epoch', profile_batch=0, embeddings_freq=0, embeddings_metadata=None, **kwargs)
    Initialize self.  See help(type(self)) for accurate signature.
